In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
#sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
#sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
#sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
#sys.path.insert(1, '/home/matthew/Documents/code/photometry_analysis_code/freely_moving_photometry_analysis')
sys.path.insert(1, 'C:\\Users\\SJLab_Slice\\Documents\\photometry_analysis\\freely_moving_photometry_analysis')

import peakutils
from matplotlib import colors, cm
from scipy.signal import decimate
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean
from utils.plotting import get_photometry_around_event
from scipy import stats
from utils.individual_trial_analysis_utils import SessionData, get_photometry_around_event, get_next_centre_poke, get_next_reward_time, HeatMapParams

In [33]:
class CustomAlignedDataPsychometric(object):
    def __init__(self, session_data, params, trial_type):
       # saving_folder = '/mnt/winstor/swc/sjones/users/francesca/photometry_2AC/processed_data/' + session_data.mouse + '/'
        saving_folder = "W:\\users\\Yvonne\\photometry_2AC\\processed_data\\" + session_data.mouse + '\\'
        restructured_data_filename = session_data.mouse + '_' + session_data.date + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = session_data.mouse + '_' + session_data.date + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)

        fiber_options = np.array(['left', 'right'])
        fiber_side_numeric = (np.where(fiber_options == session_data.fiber_side)[0] + 1)[0]
        contra_fiber_side_numeric = (np.where(fiber_options != session_data.fiber_side)[0] + 1)[0]

        self.contra_data = ZScoredTracesPsychometric(trial_data, dff,params, contra_fiber_side_numeric, contra_fiber_side_numeric, trial_type)
        self.contra_data.get_peaks()
        #self.ipsi_data = ZScoredTracesPsychometric(trial_data, dff,params, fiber_side_numeric, fiber_side_numeric, trial_type)
        #self.ipsi_data.get_peaks()

In [24]:
def get_all_experimental_records():
   # experiment_record = pd.read_csv('/mnt/winstor/swc/sjones/users/francesca/photometry_2AC/experimental_record_matt.csv')
    experiment_record = pd.read_csv("W:\\users\\Yvonne\\photometry_2AC\\experimental_record_yvonne.csv")
    experiment_record['date'] = experiment_record['date'].astype(str)
    return experiment_record

In [11]:
def open_experiment(experiment_to_add, mouse):
    for index, experiment in experiment_to_add.iterrows():
        #saving_folder = '/mnt/winstor/swc/sjones/users/francesca/photometry_2AC/processed_data/' + experiment['mouse_id'] + '/'
        
        saving_folder = "W:\\users\\Yvonne\\photometry_2AC\\processed_data\\" + mouse + '\\'
        restructured_data_filename = experiment['mouse_id'] + '_' + experiment['date'] + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = experiment['mouse_id'] + '_' + experiment['date'] + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)
        session_traces = SessionData(experiment['fiber_side'], experiment['recording_site'], experiment['mouse_id'], experiment['date'])
    return session_traces

In [5]:
def get_peak_each_trial_psychometric(sorted_traces, time_points, sorted_other_events):
    all_trials_peaks = []
    #plt.figure()
    for trial_num in range(0, len(sorted_other_events)):
        indices_to_integrate = np.where(np.logical_and(np.greater_equal(time_points, 0),
                                                       np.less_equal(time_points, sorted_other_events[trial_num])))
        trial_trace = (sorted_traces[trial_num, indices_to_integrate]).T
        trial_trace = trial_trace  # - trial_trace[0]
        trial_peak_inds = peakutils.indexes(trial_trace.flatten('F'), thres=0.3)
        if trial_peak_inds.shape[0] > 0:
            if len(trial_peak_inds > 1):
                trial_peak_inds = trial_peak_inds[0]

            trial_peaks = trial_trace.flatten('F')[trial_peak_inds]
            all_trials_peaks.append(trial_peaks)
            #plt.plot(trial_trace)
            #plt.scatter(trial_peak_inds, trial_peaks)
    flat_peaks = all_trials_peaks
    #plt.show()
    return flat_peaks

In [6]:
def get_peak_each_trial_psychometric_no_nans(sorted_traces, time_points, sorted_other_events):
    all_trials_peaks = []
    plt.figure()
    for trial_num in range(0, len(sorted_other_events)):
        indices_to_integrate = np.where(np.logical_and(np.greater_equal(time_points, 0), np.less_equal(time_points, sorted_other_events[trial_num])))
        trial_trace = (sorted_traces[trial_num, indices_to_integrate]).T
        trial_trace = trial_trace # - trial_trace[0]s
        trial_peak_inds = peakutils.indexes(trial_trace.flatten('F'), thres=0.3)
        if trial_peak_inds.shape[0] > 0 or len(trial_peak_inds > 1):
            trial_peak_inds = trial_peak_inds[0]
            trial_peaks = trial_trace.flatten('F')[trial_peak_inds]
        else:
            trial_peak_inds = np.argmax(trial_trace)
            trial_peaks = np.max(trial_trace)
            print(trial_num, trial_peak_inds, trial_peaks)
        all_trials_peaks.append(trial_peaks)
        plt.plot(trial_trace)
        plt.scatter(trial_peak_inds, trial_peaks)
    flat_peaks = all_trials_peaks
    return flat_peaks
    #plt.show()

In [7]:
def get_max_each_trial_psychometric(sorted_traces, time_points, sorted_other_events):
    all_trials_peaks = []
    #plt.figure()
    for trial_num in range(0, len(sorted_other_events)):
        indices_to_integrate = np.where(np.logical_and(np.greater_equal(time_points, 0), np.less_equal(time_points, sorted_other_events[trial_num])))
        trial_trace = (sorted_traces[trial_num, indices_to_integrate]).T
        trial_trace = trial_trace # - trial_trace[0]s

        trial_peak_inds = np.argmax(trial_trace)
        trial_peaks = np.max(trial_trace)

        all_trials_peaks.append(trial_peaks)
        #plt.plot(trial_trace)
        #plt.scatter(trial_peak_inds, trial_peaks)
    flat_peaks = all_trials_peaks
    return flat_peaks
    #plt.show()

In [8]:
class ZScoredTracesPsychometric(object):
    def __init__(self,  trial_data, dff, params, response, first_choice, trial_type):
        self.trial_peaks = None
        self.params = HeatMapParams(params, response, first_choice)
        self.trial_type = trial_type
        self.time_points, self.mean_trace, self.sorted_traces, self.reaction_times, self.state_name, title, self.sorted_next_poke, self.trial_nums, self.event_times, self.outcome_times = find_and_z_score_traces_psycho(
            trial_data, dff, self.params, self.trial_type)

    def get_peaks(self):
        self.trial_peaks = get_peak_each_trial_psychometric(self.sorted_traces, self.time_points, self.outcome_times)

In [9]:
def find_and_z_score_traces_psycho(trial_data, demod_signal, params, trial_type, norm_window=8, sort=False, get_photometry_data=True):
    response_names = ['both left and right', 'left', 'right']
    outcome_names = ['incorrect', 'correct', 'both correct and incorrect']
    events_of_int = trial_data.loc[(trial_data['State type'] == params.state)]
    if  params.state == 10:
        omission_events = trial_data.loc[(trial_data['State type'] == params.state)]
        trials_of_int = omission_events['Trial num'].values
        omission_trials_all_states = trial_data.loc[(trial_data['Trial num'].isin(trials_of_int))]
        events_of_int = omission_trials_all_states.loc[(omission_trials_all_states['State type'] == 4)]
    else:
        events_of_int = trial_data.loc[(trial_data['State type'] == params.state)]
    if params.response != 0:
        events_of_int = events_of_int.loc[events_of_int['Response'] == params.response]
    if params.first_choice != 0:
        events_of_int = events_of_int.loc[events_of_int['First response'] == params.first_choice]
    if params.last_response != 0:
        events_of_int = events_of_int.loc[events_of_int['Last response'] == params.last_response]
        title = ' last response: ' + response_names[params.last_response]
    else:
        title = response_names[params.response]
    if not params.outcome == 2: # if you don't care about the reward or not
        events_of_int = events_of_int.loc[events_of_int['Trial outcome'] == params.outcome]
    #events_of_int = events_of_int.loc[events_of_int['Last outcome'] == 0]

    if params.cue == 'high':
        events_of_int = events_of_int.loc[events_of_int['Trial type'] == 7]
    elif params.cue == 'low':
        events_of_int = events_of_int.loc[events_of_int['Trial type'] == 1]
    events_of_int = events_of_int.loc[events_of_int['Trial type'] == trial_type]



    if params.state == 10:
        title = title + ' ' + 'omission'
    else:
        title = title + ' ' + outcome_names[params.outcome]

    if params.instance == -1:
        events_of_int = events_of_int.loc[
            (events_of_int['Instance in state'] / events_of_int['Max times in state'] == 1)]
    elif params.instance == 1:
        events_of_int = events_of_int.loc[(events_of_int['Instance in state'] == 1)]
        if params.no_repeats == 1:
            events_of_int = events_of_int.loc[events_of_int['Max times in state'] == 1]
    elif params.instance == 0:
        events_of_int = events_of_int

    if params.first_choice_correct != 0:
        events_of_int = events_of_int.loc[
            (events_of_int['First choice correct'] == 1)]

    event_times = events_of_int[params.align_to].values
    trial_nums = events_of_int['Trial num'].values
    state_name = events_of_int['State name'].values[0]
    other_event = np.asarray(
        np.squeeze(events_of_int[params.other_time_point].values) - np.squeeze(events_of_int[params.align_to].values))
    outcome_times = get_next_reward_time(trial_data, events_of_int)
    outcome_times = outcome_times - event_times

    last_trial = np.max(trial_data['Trial num'])
    last_trial_num = events_of_int['Trial num'].unique()[-1]
    events_reset_indx = events_of_int.reset_index(drop=True)
    last_trial_event_indx = events_reset_indx.loc[(events_reset_indx['Trial num'] == last_trial_num)].index
    next_centre_poke = get_next_centre_poke(trial_data, events_of_int, last_trial_num==last_trial)
    
    next_centre_poke_norm = next_centre_poke - event_times

    # this all deals with getting photometry data
    if get_photometry_data == True:
        event_photo_traces = get_photometry_around_event(event_times, demod_signal, pre_window=norm_window,
                                                         post_window=norm_window)
        norm_traces = stats.zscore(event_photo_traces.T, axis=0)

        if len(other_event) != norm_traces.shape[1]:
            other_event = other_event[:norm_traces.shape[1]]
        if sort:
            arr1inds = other_event.argsort()
            sorted_other_event = other_event[arr1inds[::-1]]
            sorted_traces = norm_traces.T[arr1inds[::-1]]
            sorted_next_poke = next_centre_poke_norm[arr1inds[::-1]]
        else:
            sorted_other_event = other_event
            sorted_traces = norm_traces.T
            sorted_next_poke = next_centre_poke_norm

        time_points = np.linspace(-norm_window, norm_window, norm_traces.shape[0], endpoint=True, retstep=False, dtype=None,
                             axis=0)
        mean_trace = np.mean(sorted_traces, axis=0)

        return time_points, mean_trace, sorted_traces, sorted_other_event, state_name, title, sorted_next_poke, trial_nums, event_times, outcome_times
    else:
        if sort:
            arr1inds = other_event.argsort()
            sorted_other_event = other_event[arr1inds[::-1]]
            sorted_next_poke = next_centre_poke_norm[arr1inds[::-1]]
        else:
            sorted_other_event = other_event
            sorted_next_poke = next_centre_poke_norm
        return sorted_other_event, state_name, title, sorted_next_poke, trial_nums, event_times




In [12]:
mouse_id = 'YJ_03' #'SNL_photo57'
date = '20220524'
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
session_data = open_experiment(experiment_to_process, mouse_id)

In [14]:
#saving_folder = '/mnt/winstor/swc/sjones/users/francesca/photometry_2AC/processed_data/' + mouse_id + '/'
saving_folder = "W:\\users\\Yvonne\\photometry_2AC\\processed_data\\" + mouse_id + '\\'
demod_signal = np.load(saving_folder + mouse_id + '_' + date + "_smoothed_signal.npy" )
plt.plot(demod_signal)

In [34]:
all_trial_type_data = []
for trial_type in range(1,8):
    try:
        params = {'state_type_of_interest': 5,
            'outcome': 1,
            'last_outcome': 0,  # NOT USED CURRENTLY
            'no_repeats' : 1,
            'last_response': 0,
            'align_to' : 'Time start',
            'instance':-1,
            'plot_range': [-6, 6],
            'first_choice_correct': 1,
             'cue': 'None'}
        all_trial_type_data.append(CustomAlignedDataPsychometric(session_data, params, trial_type))
    except:
        print(trial_type)

5
6
7


In [30]:
all_trial_type_data[0]

In [35]:
colours = cm.inferno(np.linspace(0, 1, 7))
fig, ax = plt.subplots(1,2, sharey=True,sharex=True)
for trial_type in range(1,len(all_trial_type_data)+1):
    print (trial_type)
    one_side_data = all_trial_type_data[trial_type-1].contra_data
    mean_trace = decimate(one_side_data.mean_trace, 10)
    time_points = decimate(one_side_data.time_points, 10)
    traces = decimate(one_side_data.sorted_traces, 10)
    ax[0].plot(time_points, mean_trace, lw=1.5, color=colours[trial_type-1], label= trial_type)
    ax[0].axvline(0, color='k')
    ax[0].set_xlim([-0.5, 1.5])
    
    '''one_side_data = all_trial_type_data[trial_type-1].ipsi_data
    mean_trace = decimate(one_side_data.mean_trace, 10)
    time_points = decimate(one_side_data.time_points, 10)
    traces = decimate(one_side_data.sorted_traces, 10)
    ax[1].plot(time_points, mean_trace, lw=1.5, color=colours[trial_type-1], label= trial_type)
    ax[1].axvline(0, color='k')'''
plt.legend()


1
2
3


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


4


### colours = cm.inferno(np.linspace(0, 1, 7))
fig2, ax = plt.subplots(1,3)
for trial_type in range(1,8):
    one_side_data = all_trial_type_data[trial_type-1].contra_data
    x_vals = np.ones([len(one_side_data.trial_peaks)])*trial_type
    ax[0].hist(one_side_data.trial_peaks, color=colours[trial_type-1], alpha=0.4)
    ax[1].scatter(trial_type, np.mean(one_side_data.trial_peaks), color=colours[trial_type-1])
    ax[1].set_ylabel('Mean peak')
    ax[2].scatter(trial_type, np.median(one_side_data.trial_peaks), color=colours[trial_type-1])
    ax[2].set_ylabel('Median peak')
    print(np.mean(one_side_data.trial_peaks), np.median(one_side_data.trial_peaks))

In [108]:
import matplotlibtlib

In [134]:
colourmap = matplotlib.cm.viridis
colours = colourmap(np.linspace(0, 0.8, one_side_data.sorted_traces.shape[0]))
plt.figure()
one_side_data = all_trial_type_data[].contra_data
for t in range(0, one_side_data.sorted_traces.shape[0]):
    plt.plot(one_side_data.sorted_traces[t, :], color=colours[t])
    plt.axhline(one_side_data.trial_peaks[t], color=colours[t])
#plt.imshow(one_side_data.sorted_traces, aspect='auto')


[1.5414953388039705, 2.0412290735827696, 1.3379246718026432, 1.950286413924951, 0.766849123870784, 2.759143005421707, -0.7323453054153348, 1.548990558349745, 0.9185640779593433, 0.22867490960880793, 2.320498109591071, 1.6355107490045058, 0.2817006288256168, -0.25065506832706647, 2.145190902642128]


In [136]:
plt.figure()
plt.plot(one_side_data.sorted_traces[-5, :], color=colours[t])

In [105]:
plt.figure()
plt.plot(one_side_data.mean_trace)

In [217]:
np.median(np.asarray(all_trial_type_data[-2].contra_data.trial_peaks)/4)

0.15315428214827714

In [ ]:
np.median(np.asarray(all_trial_type_data[-2].contra_data.trial_peaks)/4)

In [209]:
np.asarray(one_side_data.trial_peaks)/4

array([0.14386391, 0.36703822, 0.03294286, 0.67309225, 0.1799292 ,
       0.26867563, 0.06190406])

In [199]:
np.median([0.13636681, 0.46637622, 0.51340394])

0.46637622

In [207]:
np.median(np.concatenate([np.asarray(one_side_data.trial_peaks),np.asarray([0.13636681, 0.46637622, 0.51340394])]))

0.5444297804051131

In [37]:
all_trial_type_data